In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Thu thập các đường dẫn từ URL

In [2]:
import requests
import os
from tqdm import tqdm
from bs4 import BeautifulSoup as bs
from urllib.parse import urljoin

import re

def find_links_with_document(url):
    response = requests.get(url)
    if response.status_code == 200:
        content = response.text
        pattern = r"(?=.*document).*\bhttp[s]?://\S+\b"
        links_with_document = []
        soup = bs(content, "lxml")
        links = soup.find_all("a")

        for link in links:
            href = link.get("href")
            if href:
                if re.search(pattern, href):
                    links_with_document.append(href)
                    print(href)

        return links_with_document
    else:
        print("Failed to retrieve the web page.")
        return []

# Download file HTML

In [3]:
def download_fileHTML(filename, url):
  response = requests.get(url)
  html_content = response.text
    # Đường dẫn đến thư mục bạn muốn lưu trữ file tải về
  folder_path = '/content/drive/MyDrive/XLNNTN-MSc-S2/file_html'

  # Kiểm tra xem thư mục đích có tồn tại không, nếu không thì tạo mới
  if not os.path.exists(folder_path):
    os.makedirs(folder_path)
  # Đường dẫn đến file đích
  file_path = os.path.join(folder_path, filename)

  # Lưu nội dung vào file HTML
  with open(file_path, "w", encoding='utf-8') as file:
      file.write(html_content)

  print("Tải xuống hoàn thành. Source code được lưu trong file "+filename)

# Download images

Ref: https://www.thepythoncode.com/article/download-web-page-images-python

In [4]:
from urllib.parse import urlparse

def is_valid(url):
    """
    Checks whether `url` is a valid URL.
    """
    parsed = urlparse(url)
    return bool(parsed.netloc) and bool(parsed.scheme)

def get_all_images(url):
    """
    Returns all image URLs on a single `url`
    """
    soup = bs(requests.get(url).content, "html.parser")
    #print("---soup---\n", soup)
    urls = []
    for img in tqdm(soup.find_all("img",attrs={'class':'absimg'}), "Extracting images"):
        img_url = img.attrs.get("src") or img.attrs.get("data-src") or img.attrs.get("data-original") or img.attrs.get("orig")
        print(img_url)
        if not img_url:
            # if img does not contain src attribute, just skip
            continue
        # make the URL absolute by joining domain with the URL that is just extracted
        img_url = urljoin(url, img_url)
        try:
            pos = img_url.index("?")
            img_url = img_url[:pos]
        except ValueError:
            pass
        # finally, if the url is valid
        if is_valid(img_url):
            urls.append(img_url)
    return urls

def download(url, pathname):
    """
    Downloads a file given an URL and puts it in the folder `pathname`
    """
    # if path doesn't exist, make that path dir
    if not os.path.isdir(pathname):
        os.makedirs(pathname)
    # download the body of response by chunk, not immediately
    response = requests.get(url, stream=True)
    # get the total file size
    file_size = int(response.headers.get("Content-Length", 0))
    # get the file name
    filename = os.path.join(pathname, url.split("/")[-1])
    # progress bar, changing the unit to bytes instead of iteration (default by tqdm)
    progress = tqdm(response.iter_content(1024), f"Downloading {filename}", total=file_size, unit="B", unit_scale=True, unit_divisor=1024)
    with open(filename, "wb") as f:
        for data in progress.iterable:
            # write data read to the file
            f.write(data)
            # update the progress bar manually
            progress.update(len(data))




# Download Text

## Get Div's Style Attribute (Top, Left)

In [5]:
def GetStyle_Attribute(tags):

  # Kiểm tra xem thẻ div có thuộc tính style hay không
  if 'style' in tags.attrs:
      # Lấy giá trị của thuộc tính style
      style_value = tags['style']

      # Tách các cặp key-value trong thuộc tính style
      style_pairs = style_value.split(';')

      # Tìm giá trị của "top" và "left"
      top_value = None
      left_value = None

      for pair in style_pairs:
          key_value = pair.split(':', 1)
          if len(key_value) == 2:
              key = key_value[0].strip()
              value = key_value[1].strip()
              if key == 'top':
                  top_value = int(value.rstrip('px'))
              elif key == 'left':
                  left_value = int(value.rstrip('px'))
      return top_value, left_value
  else:
      return None, None


## Read and write back to text file

In [6]:
import re
from requests.auth import HTTPBasicAuth
def get_all_text(source, filename):

    # Đường dẫn đến thư mục bạn muốn lưu trữ file tải về
    folder_path = '/content/drive/MyDrive/XLNNTN-MSc-S2/Text'

    # Kiểm tra xem thư mục đích có tồn tại không, nếu không thì tạo mới
    if not os.path.exists(folder_path):
      os.makedirs(folder_path)
    # Đường dẫn đến file đích
    file_path = os.path.join(folder_path, filename + '.txt')


    # Đọc nội dung từ file HTML
    path_htmlfile = source +'/'+ filename + '.html'
    with open(path_htmlfile, 'r') as file:
        html_content = file.read()
        print("đọc thành công")

    soup = bs(html_content, 'html.parser')
    content = soup.find('div', class_='outer_page_container')

    # Lọc các thẻ trên file HTML
    tags = content.find_all(['div', 'span'])

    # In ra nội dung của các thẻ
    # Start_top là mức trung bình giữa các kí tự trên cùng 1 hàng
    with open(file_path, 'w', encoding='utf-8') as file:
      top, left = GetStyle_Attribute(tags[0])
      start_top = top or 0

      for tag in tags:
        if tag.text and not re.match(r'^\n', tag.text):
          top, left = GetStyle_Attribute(tag)
          top = top or start_top

          if(len(tag.find_all(recursive=False)) > 0):
            if((top - start_top > 19 or (top - start_top) < -90) and tag.get_text() != " " and tag.get_text() != None ):
              file.write( "\n"+str(top)+";"+str(left)+";"+str(start_top)+";")
              start_top = top
            continue
          elif((top - start_top > 19 or (top - start_top) < -90) and tag.get_text() != " " and tag.get_text() != None ):
            file.write( "\n"+str(top)+";"+str(left)+";"+str(start_top)+";"+tag.get_text())
            start_top = top
          elif(top - start_top < 19 and tag.get_text() != " " and tag.get_text() != None ):
            #file.write( "\n"+"--- " +str(top)+ " --- "+str(start_top))
            file.write(tag.get_text())
            if(top > start_top):
              start_top =(start_top + top)/2



## Check String

### Check String is numberic, alphabet,... or not

In [7]:
def check_string(string):
    count = 0
    result = 1
    if(len(string) > 0):
      for char in string:

        # Kiểm tra số
        if re.search(r'\d', char):
            count = count + 1
            #print(str(count)+" "+ char)

        # Kiểm tra chữ cái
        if re.search(r'[a-zA-Z]', char):
            count = count + 1
            #print(str(count)+" "+ char)

        # Kiểm tra kí tự đặc biệt
        if re.search(r'\W', char):
            count = count + 1
            #print(str(count)+" "+ char)

      result = count/len(string)
    return result


### Check the match between 2 strings

In [8]:
!pip install python-Levenshtein

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.5/172.5 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 12.0 MB/s eta 0:00:00


In [9]:
import Levenshtein

def calculate_similarity(string1, string2):
    distance = Levenshtein.distance(string1, string2)
    max_length = max(len(string1), len(string2), 1)
    similarity = (max_length - distance) / max_length * 100
    return similarity

string1 = " "
string2 = ""

similarity_percentage = calculate_similarity(string1, string2)
print(f"Similarity: {similarity_percentage}%")


Similarity: 0.0%


In [10]:
def compare_remaining(string_A, string_B):
    strA = string_A.lower().strip().replace(" ", "")
    strB = string_B.lower().strip().replace(" ", "")

    if len(strA) > len(strB):
      temp = strA
      strA = strB
      strB = temp
    length_A = len(strA)
    length_B = len(strB)
    match_ratio = 0.0
    for i in range(0,length_B - length_A + 1):
      extracted = strB[i:length_A + i]
      match_ratio_extracted = calculate_similarity(extracted, strA)
      if(match_ratio_extracted < match_ratio):
        break
      else:
        match_ratio = match_ratio_extracted
    return float(match_ratio)

# string = "九重按劍起當席"
# # Ví dụ sử dụng
# chuoi_A = hcmus_translate(string)
# chuoi_B = "cửu trùng án kiếm khởi đương tịch "

# compare_remaining(chuoi_A, chuoi_B)


### Call HCMUS_Traslatetor API

In [11]:
import json
import time

def hcmus_translate(text):
    url = 'https://api.clc.hcmus.edu.vn/nom_translation/90/1'
    response = requests.request('POST', url, data={'nom_text': text})
    time.sleep(0.1)

    try:
        result = json.loads(response.text)['sentences']
        result = result[0][0]['pair']['modern_text']
        return result
    except:
        print(f'[ERR] "{text}": {response.text}')
        return 'Cannot translate this text.'

In [31]:
hcmus_translate("meo 寂寞空閨寶篆殘")

'm e o tịch mịch không khuê bảo triện tàn '

### Call HVDic_Traslatetor API

In [13]:
def hvdic_translate(text):
  count_Nom_Character = 0
  output = ""
  meaning = []
  if(text is not None):
    url = 'https://hvdic.thivien.net/transcript-query.json.php'
    headers = { 'Content-Type': 'application/x-www-form-urlencoded; charset=  ' }

    # Request phonetics for Hán Việt (lang=1) first, if the response result is not
    # Hán Việt (contains blank lists) => Request phonetics for Nôm (lang=3)
    for lang in [1, 3]:
        payload = f'mode=trans&lang={lang}&input={text}'
        response = requests.request('POST', url, headers=headers, data=payload.encode())
        time.sleep(0.1)
        try:

            result = json.loads(response.text)['result']
        except:
            print(f'[ERR] {text}: {response.text}')
            result = {}
    for phonetics_dict in result:
      if phonetics_dict['t'] == 3 and len(phonetics_dict['o']) >= 0:
        output += phonetics_dict['i']
        meaning.append(phonetics_dict['o'])
        count_Nom_Character += 1
  return output, meaning, count_Nom_Character

In [30]:
hvdic_translate("無")

ConnectTimeout: ignored

In [14]:
from concurrent.futures import ThreadPoolExecutor

def remove_unprintable_characters(input_string):
    printable_chars = [char for char in input_string if char.isprintable()]
    return ''.join(printable_chars)

def remove_duplicates_from_B(A, B):
    unique_chars_A = set(A)
    new_B = ''
    if B is not None:
      B = remove_unprintable_characters(B)
      new_B = [char for char in B if char not in unique_chars_A]
    return ''.join(new_B)

def parallel_remove_duplicates(A, B):
    with ThreadPoolExecutor() as executor:
        future_B = executor.submit(remove_duplicates_from_B, A, B)

    result_B = future_B.result()

    return result_B

In [15]:
def check_String_Match(string, Nom_String, Meaning):
  VN_String = parallel_remove_duplicates(Nom_String, string)
  VN_Character = VN_String.split(' ')
  count_VN_Character = len(VN_Character)-1
  count = 0
  String_Match = ''
  Rate = 0
  if(len(Nom_String)>0):
    for char in VN_Character:
      char =  char.lower().strip()
      for mean in Meaning:
        for m in mean:
          if(calculate_similarity(m,char) >= 70):
            count += 1
            String_Match += char + " "
            Meaning.remove(mean)
            break

    Rate = count/len(Nom_String)
  Label = None
  if(Rate >= 0.75):
    Label = 'Reliable'
  elif(Rate >= 0.60):
    Label ='Trustable'
  elif(Rate >= 0.50):
    Label = 'Needs_verification'
  else:
    Label = 'Unknown'
  return Label, String_Match, Rate

In [16]:
# string = remove_unprintable_characters("整篆庫来台 chễnh")

# Nom_String, Meaning, count_Nom_Character = hvdic_translate(string)
# Label, String_Match, rate = check_String_Match(string, Nom_String, Meaning)
# print(rate)
# print(count_Nom_Character)
# print(String_Match)
# print(Label)


## Generate Excel File

### Decorate cells

In [17]:
def FillColor_Cell(cell, type):
  color_mapping = {
  'Reliable': 'CCFFCC',  # Xanh nhạt
  'Trustable': 'FFFFCC',  # Vàng nhạt
  'Needs_verification': 'FFCCCC',  # Đỏ nhat
  'Unknown': 'FF0000', # Đỏ đậm
  'Heading':'FFFF00'
  }
  cell.fill = PatternFill(start_color=color_mapping[type], end_color=color_mapping[type], fill_type="solid")


### Format and Border table

In [18]:
from openpyxl.styles import Border, Side
from openpyxl.utils import get_column_letter

def FormatTable(sheet):
  for col in sheet.columns:
    max_length = 0
    column = col[0].column_letter
    for cell in col:
        try:
            if len(str(cell.value)) > max_length:
                max_length = len(cell.value)
        except:
            pass
    adjusted_width = (max_length + 2) * 1.2
    sheet.column_dimensions[column].width = adjusted_width

  # Định dạng viền cho ô
  border = Border(left=Side(style='thin'),
                  right=Side(style='thin'),
                  top=Side(style='thin'),
                  bottom=Side(style='thin'))

  for row in sheet.iter_rows():
    if any(cell.value is not None for cell in row):
      for cell in row:
          cell.border = border

### Create Heading

In [19]:
def CreateHeading(sheet):
  sheet["A1"] = "Top"
  sheet["B1"] = "Left"
  sheet["C1"] = "Start_top"
  sheet["D1"] = "Text"
  sheet["E1"] = "Translated by Document"
  sheet["F1"] = "Translated by Dictionary"
  FillColor_Cell(sheet["A1"],'Heading')
  FillColor_Cell(sheet["B1"],'Heading')
  FillColor_Cell(sheet["C1"],'Heading')
  FillColor_Cell(sheet["D1"],'Heading')
  FillColor_Cell(sheet["E1"],'Heading')
  FillColor_Cell(sheet["F1"],'Heading')

### Write in Excel file

In [20]:
def WriteExcelFile(sheet, filetext_path):
  # Mở file văn bản và đọc từng dòng
  with open(filetext_path, "r") as file:
    for line in file:
      # Tách các thuộc tính trong dòng bằng dấu ";"
      attributes = line.strip().split(";")

      # Ghi các thuộc tính vào các cell tương ứng trong cùng một hàng
      sheet.append(attributes)

### Separate Nôm text and Vietnamese Text

In [21]:
check_string("ẠCH VÂN AM QUỐC NGỮ")

0.7894736842105263

In [22]:
def SeparateNomText(sheet):
  #row = 1
  # Check string and call api to translate
  for row in range(2, len(sheet["C"])):
    string = sheet.cell(row=row, column=4).value
    # Translate
    Nom_String, Meaning, count_Nom_Character = hvdic_translate(string)
    Label, String_Match, Rate = check_String_Match(string, Nom_String, Meaning)
    VN_String = parallel_remove_duplicates(Nom_String, string)
    value_to_search = ""
    if(count_Nom_Character > 0 and Rate == 0):
      value_to_search = hcmus_translate(sheet.cell(row=row, column=4).value)
      #print(value_to_search)


    elif(count_Nom_Character > 0 and Rate > 0.1):
      sheet.cell(row=row, column=6).value = hcmus_translate(Nom_String)

      if(Label != 'None'):
        sheet.cell(row=row, column=5).value = String_Match + "---in---[Original] " + VN_String
      if(Label == 'Reliable'):
        FillColor_Cell(sheet.cell(row=row, column=5),'Reliable')
      elif(Label == 'Trustable'):
        FillColor_Cell(sheet.cell(row=row, column=5),'Trustable')
      elif(Label == 'Needs_verification'):
        FillColor_Cell(sheet.cell(row=row, column=5),'Needs_verification')
      elif(Label == 'Unknown'):
        FillColor_Cell(sheet.cell(row=row, column=5),'Unknown')
      sheet.cell(row=row, column=4).value = Nom_String

    elif(count_Nom_Character == 0):
      continue


    for row_j in range(2, len(sheet["C"])):
      if(value_to_search == ""):
        break
      if(sheet.cell(row=row_j, column=4).value is not None):
        match_rate = compare_remaining(value_to_search, str(sheet.cell(row=row_j, column=4).value))
        if match_rate >= 75.00:
          # Ghi giá trị tìm được vào cell D4
          if(sheet.cell(row=row, column=5).value is not None):
            sheet.cell(row=row, column=5).value += sheet.cell(row=row_j, column=4).value
          else:
            sheet.cell(row=row, column=5).value = sheet.cell(row=row_j, column=4).value
            FillColor_Cell(sheet.cell(row=row, column=5),'Reliable')
          sheet.delete_rows(row_j)
        elif match_rate >= 60.00:
          # Ghi giá trị tìm được vào cell D4
          if(sheet.cell(row=row, column=5).value is None):
            sheet.cell(row=row, column=5).value = sheet.cell(row=row_j, column=4).value
            FillColor_Cell(sheet.cell(row=row, column=5),'Trustable')
            sheet.delete_rows(row_j)
        elif match_rate >= 52.00:
          # Ghi giá trị tìm được vào cell D4
          if(sheet.cell(row=row, column=5).value is None):
            sheet.cell(row=row, column=5).value = sheet.cell(row=row_j, column=4).value
            FillColor_Cell(sheet.cell(row=row, column=5),'Needs_verification')
            sheet.delete_rows(row_j)
      sheet.cell(row=row, column=6).value = value_to_search

In [23]:
import openpyxl
from openpyxl.styles import PatternFill
from openpyxl import Workbook
def output_Excel(filename):
  folder_path = '/content/drive/MyDrive/XLNNTN-MSc-S2/Text'
  filetext_path = os.path.join(folder_path, filename + '.txt')
  file_path_excel = os.path.join(folder_path, filename + '.xlsx')
  workbook = openpyxl.Workbook()
  sheet = workbook.active

  # Create Heading
  CreateHeading(sheet)
  # Write Excel file from file text
  WriteExcelFile(sheet, filetext_path)
  # Separate Text
  SeparateNomText(sheet)

  # Format Table
  FormatTable(sheet)

  filename = file_path_excel
  workbook.save(filename)

In [24]:
# output_Excel("Bạch-Van-Am-Quốc-Ngữ-Nguyễn-Bỉnh-Khiem")

# Main

In [27]:
def main(url, path):
    links_with_document = find_links_with_document(url=url)
    print(links_with_document.count)
    if links_with_document:
        print("Các liên kết chứa từ 'document':")

        # source_fileHTML = '/content/drive/MyDrive/XLNNTN-MSc-S2/Results/Constant_HTLM_file'
        source_fileHTML = '/content/drive/MyDrive/MSc_S2/XLNNTN-MSc-S2/Results/Constant_HTLM_file'
        for link in links_with_document:
            filename = str(link).split('/')[-1]
            images = get_all_images(link)
            download_fileHTML(filename + '.html',link)
            get_all_text(source_fileHTML, filename)
            output_Excel(filename)
            for img in images:
              download(img, path)

    else:
        print("Không tìm thấy liên kết chứa từ 'document'.")

# Run

In [29]:
if __name__ == '__main__':
    main("https://www.scribd.com/lists/21643875/Th%C6%A1-V%C4%83n-Han-Nom","/content/drive/MyDrive/XLNNTN-MSc-S2/images")

https://www.scribd.com/document/354356904/Thập-Giới-Co-Hồn-Quốc-Ngữ-Văn-Le-Thanh-Tong
https://www.scribd.com/document/351003106/Thien-Nam-Ngữ-Lục-Thơ-Nom
https://www.scribd.com/document/354358928/Cung-Oan-Ngam-Khuc-Nguyễn-Gia-Thiều-Le-Văn-Đặng-Diễn-Nom
https://www.scribd.com/document/354358921/Chinh-Phụ-Ngam-Diễn-Ca-Nguyễn-Văn-Xuan
https://www.scribd.com/document/354359286/Chinh-Phụ-Ngam-Han-Việt
https://www.scribd.com/document/351002527/Ma-Thanh-Tan-Truyện-Lý-Hồng-Phượng
https://www.scribd.com/document/354356908/Bạch-Van-Am-Quốc-Ngữ-Nguyễn-Bỉnh-Khiem
https://www.scribd.com/document/354356906/Bai-Tựa-Đại-Nam-Quốc-Ngữ
https://www.scribd.com/document/354361314/Minh-Tam-Bửu-Giam-Diễn-Nom
https://www.scribd.com/document/354361116/Nữ-Tử-Tu-Tri-Diễn-Nom
https://www.scribd.com/document/354361115/Mộng-Lien-Đinh-Thi-Thảo-Nguyễn-Gia-Tuyển
https://www.scribd.com/document/354359991/Giao-Tử-Phu-Mạc-Đĩnh-Chi
https://www.scribd.com/document/354359995/Hồ-Xuan-Hương-Thi-Tập-Le-Văn-Đặng-Diễn-Nom
https:/

Extracting images: 0it [00:00, ?it/s]


Tải xuống hoàn thành. Source code được lưu trong file Thập-Giới-Co-Hồn-Quốc-Ngữ-Văn-Le-Thanh-Tong.html
đọc thành công


ConnectTimeout: ignored